Reference: J.J. Hopfield, Neural networks and physical systems with emergent collective computational abilities., Proc. Natl. Acad. Sci. U.S.A. 79 (8) 2554-2558, https://doi.org/10.1073/pnas.79.8.2554 (1982).

Set-up:

1. Network structure：
   * The network has $N$ neurons in the whole network.
   * The state is either 0 or 1(which is binary).

3. Connectivity matrix：
   * Hebbian learning rule：
     $$
     W_{ij} = \sum_{\ell=1}^K (2p_i^{(\ell)} - 1)(2p_j^{(\ell)} - 1)
     $$
   * And no self-connectivity：$W_{ii} = 0$

4. Recall dynamics：
   * Asynchronous；
   * The update time is drawn from exponential distribution.
   * Update if:
     $$
     \sum_{j=1}^K W_{ij}r_j >0
     $$

5. Other notation:
   * K is the number of memories stored in the network.

In [1]:
%matplotlib auto
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
from scipy.stats import expon
import random

Using matplotlib backend: Qt5Agg


In [2]:
def generate_p(N,K,probability):
    return np.reshape(bernoulli.rvs(probability,size=N*K),(N,K))

In [3]:
def generate_network(p):
    W=np.dot((2*p-1),(2*p-1).T)
    return W-np.diag(np.diag(W))

In [4]:
def real_time_plot(axe,r):
    plt.cla()
    plt.tight_layout()
    axe.set_title('r after the recall dynamics')
    axe.eventplot(np.nonzero(r))
    plt.pause(0.001)

In [5]:
def recall_dynamics(N,W,r,t_max,axe,p):
    #generate the update time
    t=np.reshape(expon.rvs(probability,size=N*25),(25,N))
    t=np.cumsum(t,axis=0)
    #sort the update time by index
    index=(np.unravel_index(np.argsort(t,axis=None),t.shape))
    t_idx=index[0]
    n_idx=index[1]
    k=0
    while(t[t_idx[k]][n_idx[k]]<=t_max):
        if (np.dot(r,W[n_idx[k],:])>0):
            r[n_idx[k]]=1
        else:
            r[n_idx[k]]=0
        real_time_plot(axe,r)
        k+=1
        if((k==25*N-1)or(p[:,0]==r).all()):
            break
    return r

In [6]:
K=5
N=200
probability=0.5
t_max=10
fraction=0.1

In [7]:
#generate the memory p and network W
p=generate_p(N,K,probability)
W=generate_network(p)

In [8]:
#initialize the corruputed version r with original p0
r=p[:,0].copy()

#randomly choose which neuron's activity is corrupted
flip=random.sample(range(0,N),int(fraction*N))

#change the activity in r
for i in range(int(fraction*N)):
    if (r[flip[i]]==0):
        r[flip[i]]=1
    else:
        r[flip[i]]=0
        

#show the three pattern in eventplot

fig,ax=plt.subplots(3,1)
ax[0].eventplot(np.nonzero(r))
ax[0].set_title('the corrupted vesion r')
ax[1].eventplot(np.nonzero(p[:,0]))
ax[1].set_title('the original memory pattern p')

#activate the interactive plot
plt.ion()

#do the recall procedure
r=recall_dynamics(N,W,r,t_max,ax[2],p)

plt.ioff()
plt.show()

#check if the memory is recalled
if ((p[:,0]==r).all()):
    print('Successfully recalled.')
else:
    print('Failed to recall.')

Successfully recalled.
